In [9]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os

In [10]:
from IPython.display import JSON

In [11]:
from googleapiclient.discovery import build

In [12]:
load_dotenv('key_youtube.env')

True

In [30]:
api_key = os.getenv('api_key')

In [31]:
channel_ids = ['UC7zsxKqd5MicTf4VhS9Y74g', 
               'UC7_YxT-KID8kRbqZo7MyscQ', 
               'UCL_Zrh9lwoOT7ftTRA-npvg', 
               'UCYCixziQqloyIQzxvsGHheg', 
               'UCYzPXprvl5Y-Sf0g4vX-m6g']

In [32]:
api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client
youtube = build(api_service_name, api_version, developerKey=api_key)

In [16]:
def get_channel_stats(youtube, channel_ids):
    
    all_data = []
    
    request = youtube.channels().list(
        part='snippet,contentDetails,statistics',
        id=','.join(channel_ids)
    )
    response = request.execute()
    
    #loop through items
    for item in response['items']:
        data = {'channelName': item['snippet']['title'],
                'subscribers': item['statistics']['subscriberCount'],
               'views': item['statistics']['viewCount'],
               'totalVideos': item['statistics']['videoCount'],
               'playlistID': item['contentDetails']['relatedPlaylists']['uploads']
               }
        
        all_data.append(data)
        
    return(pd.DataFrame(all_data))

In [17]:
channels_stats = get_channel_stats(youtube, channel_ids)

In [18]:
channels_stats

,channelName,subscribers,views,totalVideos,playlistID
0,Kooleen,407000,18606263,53,UUL_Zrh9lwoOT7ftTRA-npvg
1,Jake Munro,472000,40313790,479,UUYCixziQqloyIQzxvsGHheg
2,Markiplier,32500000,17974714379,5126,UU7_YxT-KID8kRbqZo7MyscQ
3,Kurtis Conner,3800000,508428701,197,UU7zsxKqd5MicTf4VhS9Y74g
4,jacksepticeye,28200000,15112116262,4915,UUYzPXprvl5Y-Sf0g4vX-m6g


In [19]:
mrklist_id = 'UU7_YxT-KID8kRbqZo7MyscQ'

def get_video_ids(youtube, mrklist_id):
    
    mrkvideo_ids = []
    
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=mrklist_id,
        maxResults = 50
    )
    response = request.execute()
    
    for item in response['items']:
        mrkvideo_ids.append(item['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=mrklist_id,
            maxResults = 50,
            pageToken = next_page_token)
        response = request.execute()
        
        for item in response['items']:
            mrkvideo_ids.append(item['contentDetails']['videoId'])
        
        next_page_token = response.get('nextPageToken')
        
    return mrkvideo_ids

In [20]:
mrkvid_ids = get_video_ids(youtube, mrklist_id)

In [21]:
mrkvid_ids[0:5]

['bJfoJbyHAaA', 'S_WrzagTJ3o', 'j64oZLF443g', '7bXWtXxLzlk', 'GXJiEfWwztU']

In [22]:
len(mrkvid_ids)

5126

In [23]:
def get_video_details(youtube, mrkvid_ids, start, finish):

    all_video_info = []

    request = youtube.videos().list(
        part="snippet, contentDetails, statistics",
        id=mrkvid_ids[start:finish]
        )
    response = request.execute()

    for video in response['items']:
        stats_to_keep = {'snippet': ['channelTitle', 'Title', 'description', 'tags', 'publishedAt'],
                        'statistics': ['viewCount', 'likeCount', 'favoriteCount', 'commentCount'],
                        'contentDetails': ['duration', 'definition', 'caption']
                        }
        video_info = {}
        video_info['video_id'] = video['id']

        for k in stats_to_keep.keys():
            for v in stats_to_keep[k]:
                try:
                    video_info[v] = video[k][v]
                except:
                    video_info[v] = None

        all_video_info.append(video_info)

    return pd.DataFrame(all_video_info)

In [33]:
def get_video_detailsdef(youtube, mrkvid_ids):

    all_video_info = []
    
    for i in range(0, len(mrkvid_ids), 50):
        request = youtube.videos().list(
            part="snippet, contentDetails, statistics",
            id=','.join(mrkvid_ids[i:i+50])
            )
        response = request.execute()

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'Title', 'description', 'tags', 'publishedAt'],
                            'statistics': ['viewCount', 'likeCount', 'favoriteCount', 'commentCount'],
                            'contentDetails': ['duration', 'definition', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)

    return pd.DataFrame(all_video_info)

In [25]:
#posso fazer um ','.join[i:i+50]????????? (NA LINHA 4/5)
#for i in range(0, len(mrkvid_ids), 50):
    #request = youtube.videos().list(
        #part="snippet, contentDetails, statistics",
        #id=','.join(mrkvid_ids[i:i+50])
        #)
    #response = request.execute()

In [34]:
video_df = get_video_detailsdef(youtube, mrkvid_ids)
video_df

,video_id,channelTitle,Title,description,tags,publishedAt,viewCount,likeCount,favoriteCount,commentCount,duration,definition,caption
0,bJfoJbyHAaA,Markiplier,None,I Smash and/or Pass MatPat's Perfect Pokémon. ...,"[Pokémon, Pokemon, Markiplier, Smash or pass, ...",2022-04-12T19:00:09Z,699909,140822,0,5641,PT40S,hd,false
1,S_WrzagTJ3o,Markiplier,None,ERROR: INVC_FINAL002.SHP CORRUPTED\nITERATING....,"[in space with markiplier, markiplier, iswm, s...",2022-04-11T19:00:06Z,1166813,130087,0,6888,PT1M15S,hd,false
2,j64oZLF443g,Markiplier,None,"Good to see you again, Captain!\nPut Out the F...","[markiplier, mark fischbach, markipliergame, i...",2022-04-04T19:00:11Z,7935672,466941,0,35920,PT7M7S,hd,true
3,7bXWtXxLzlk,Markiplier,None,Do you have what it takes to survive in the An...,"[markiplier, assessment examination, alternate...",2022-03-16T21:19:26Z,3195482,172262,0,6303,PT24M8S,hd,false
4,GXJiEfWwztU,Markiplier,None,The Video ► https://www.youtube.com/watch?v=QD...,"[birthday, in space with markiplier, same vide...",2022-03-14T00:42:56Z,8137156,347277,0,11716,PT2H9M29S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5121,fZAuNRIuMfc,Markiplier,None,"If you enjoyed the video, hit the LIKE button!...","[Cry of Fear, Cry of Fear Reactions, Lets Play...",2012-05-26T21:56:13Z,1094439,13553,0,867,PT3M15S,hd,false
5122,PBTpAYLktAk,Markiplier,None,"If you enjoyed the video, please hit the LIKE ...","[Amnesia Reaction Compilation, Amnesia: The Da...",2012-05-26T21:52:25Z,951233,11681,0,661,PT2M15S,hd,false
5123,7p_SxHkjSmM,Markiplier,None,"If you enjoyed the video, hit the LIKE button!...","[Nightmare House 2, Penumbra Reactions, Lets P...",2012-05-26T21:48:40Z,1223247,15256,0,950,PT3M25S,hd,false
5124,76B0DSst2ns,Markiplier,None,"If you enjoyed the video, please hit the LIKE ...","[Penumbra Reaction Compilation, Penumbra: Over...",2012-05-26T21:44:52Z,925369,11372,0,816,PT4M45S,hd,false


In [49]:
video_df.to_excel('mark_again.xlsx', index=False, encoding='UTF-8')

In [36]:
video_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5126 entries, 0 to 5125
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   video_id       5126 non-null   object
 1   channelTitle   5126 non-null   object
 2   Title          0 non-null      object
 3   description    5126 non-null   object
 4   tags           5077 non-null   object
 5   publishedAt    5126 non-null   object
 6   viewCount      5126 non-null   object
 7   likeCount      5126 non-null   object
 8   favoriteCount  5126 non-null   object
 9   commentCount   5125 non-null   object
 10  duration       5126 non-null   object
 11  definition     5126 non-null   object
 12  caption        5126 non-null   object
dtypes: object(13)
memory usage: 520.7+ KB


In [39]:
video_df = video_df.drop(columns=['Title', 'favoriteCount'])

In [40]:
video_df

,video_id,channelTitle,description,tags,publishedAt,viewCount,likeCount,commentCount,duration,definition,caption
0,bJfoJbyHAaA,Markiplier,I Smash and/or Pass MatPat's Perfect Pokémon. ...,"[Pokémon, Pokemon, Markiplier, Smash or pass, ...",2022-04-12T19:00:09Z,699909,140822,5641,PT40S,hd,false
1,S_WrzagTJ3o,Markiplier,ERROR: INVC_FINAL002.SHP CORRUPTED\nITERATING....,"[in space with markiplier, markiplier, iswm, s...",2022-04-11T19:00:06Z,1166813,130087,6888,PT1M15S,hd,false
2,j64oZLF443g,Markiplier,"Good to see you again, Captain!\nPut Out the F...","[markiplier, mark fischbach, markipliergame, i...",2022-04-04T19:00:11Z,7935672,466941,35920,PT7M7S,hd,true
3,7bXWtXxLzlk,Markiplier,Do you have what it takes to survive in the An...,"[markiplier, assessment examination, alternate...",2022-03-16T21:19:26Z,3195482,172262,6303,PT24M8S,hd,false
4,GXJiEfWwztU,Markiplier,The Video ► https://www.youtube.com/watch?v=QD...,"[birthday, in space with markiplier, same vide...",2022-03-14T00:42:56Z,8137156,347277,11716,PT2H9M29S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...
5121,fZAuNRIuMfc,Markiplier,"If you enjoyed the video, hit the LIKE button!...","[Cry of Fear, Cry of Fear Reactions, Lets Play...",2012-05-26T21:56:13Z,1094439,13553,867,PT3M15S,hd,false
5122,PBTpAYLktAk,Markiplier,"If you enjoyed the video, please hit the LIKE ...","[Amnesia Reaction Compilation, Amnesia: The Da...",2012-05-26T21:52:25Z,951233,11681,661,PT2M15S,hd,false
5123,7p_SxHkjSmM,Markiplier,"If you enjoyed the video, hit the LIKE button!...","[Nightmare House 2, Penumbra Reactions, Lets P...",2012-05-26T21:48:40Z,1223247,15256,950,PT3M25S,hd,false
5124,76B0DSst2ns,Markiplier,"If you enjoyed the video, please hit the LIKE ...","[Penumbra Reaction Compilation, Penumbra: Over...",2012-05-26T21:44:52Z,925369,11372,816,PT4M45S,hd,false


In [42]:
import isodate

In [45]:
isodate.parse_duration('PT3M25S').total_seconds()

205.0

In [47]:
video_df['duration'] = video_df['duration'].apply(lambda x: isodate.parse_duration(x).total_seconds())

In [48]:
video_df

,video_id,channelTitle,description,tags,publishedAt,viewCount,likeCount,commentCount,duration,definition,caption
0,bJfoJbyHAaA,Markiplier,I Smash and/or Pass MatPat's Perfect Pokémon. ...,"[Pokémon, Pokemon, Markiplier, Smash or pass, ...",2022-04-12T19:00:09Z,699909,140822,5641,40.0,hd,false
1,S_WrzagTJ3o,Markiplier,ERROR: INVC_FINAL002.SHP CORRUPTED\nITERATING....,"[in space with markiplier, markiplier, iswm, s...",2022-04-11T19:00:06Z,1166813,130087,6888,75.0,hd,false
2,j64oZLF443g,Markiplier,"Good to see you again, Captain!\nPut Out the F...","[markiplier, mark fischbach, markipliergame, i...",2022-04-04T19:00:11Z,7935672,466941,35920,427.0,hd,true
3,7bXWtXxLzlk,Markiplier,Do you have what it takes to survive in the An...,"[markiplier, assessment examination, alternate...",2022-03-16T21:19:26Z,3195482,172262,6303,1448.0,hd,false
4,GXJiEfWwztU,Markiplier,The Video ► https://www.youtube.com/watch?v=QD...,"[birthday, in space with markiplier, same vide...",2022-03-14T00:42:56Z,8137156,347277,11716,7769.0,hd,false
...,...,...,...,...,...,...,...,...,...,...,...
5121,fZAuNRIuMfc,Markiplier,"If you enjoyed the video, hit the LIKE button!...","[Cry of Fear, Cry of Fear Reactions, Lets Play...",2012-05-26T21:56:13Z,1094439,13553,867,195.0,hd,false
5122,PBTpAYLktAk,Markiplier,"If you enjoyed the video, please hit the LIKE ...","[Amnesia Reaction Compilation, Amnesia: The Da...",2012-05-26T21:52:25Z,951233,11681,661,135.0,hd,false
5123,7p_SxHkjSmM,Markiplier,"If you enjoyed the video, hit the LIKE button!...","[Nightmare House 2, Penumbra Reactions, Lets P...",2012-05-26T21:48:40Z,1223247,15256,950,205.0,hd,false
5124,76B0DSst2ns,Markiplier,"If you enjoyed the video, please hit the LIKE ...","[Penumbra Reaction Compilation, Penumbra: Over...",2012-05-26T21:44:52Z,925369,11372,816,285.0,hd,false
